# Dreambooth with Multiple Classes for Sunglasses

In [1]:
# # need to run ``aws configure`` to set up credentials
# import subprocess
# subprocess.call(['aws', 's3', 'cp', 
#                  's3://379552636459-stable-diffusion/experiment9-fonts/res',
#                  '../../res/', '--recursive'])

In [2]:
! pip install git+https://github.com/ShivamShrirao/diffusers.git
#! pip install -U --pre triton 
! pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/ShivamShrirao/diffusers.git to /tmp/pip-req-build-_drwjy1o
  Running command git clone --filter=blob:none --quiet https://github.com/ShivamShrirao/diffusers.git /tmp/pip-req-build-_drwjy1o
  Resolved https://github.com/ShivamShrirao/diffusers.git to commit 211dcc8d83be05707e4dd89dc008ba671ba82967
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 kB 109.7 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.8.0.dev0-py3-none-any.whl size=371315 sha256=aa89c3ba8fe4afb99e0c8d5076fd8602e23fdc4a133d3263deb6815b44157c36
  Stored in directory: /tmp/pip-ephem-wheel-cache-6yzhjiwb/wheels/8e/3c/2a/2abed2ff26b3522e046b7e52f039f821d4f652cc0ce3029fa0
Successful

In [5]:
! pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

NVIDIA A100-SXM4-40GB, 40960 MiB, 40396 MiB


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## Install Requirements

In [7]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py

In [9]:
#@title Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

### Install xformers from precompiled wheel.

## Settings and run

# Start Training

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

remove `--train_text_encoder` flag to reduce memory usage further, degrades output quality.

In [12]:
# # You can also add multiple concepts here. Try tweaking `--max_train_steps` accordingly.

# concepts_list = [
#     {
#         "instance_prompt":      "photo of zwx dog",
#         "class_prompt":         "photo of a dog",
#         "instance_data_dir":    "/content/data/zwx",
#         "class_data_dir":       "/content/data/dog"
#     },
# #     {
# #         "instance_prompt":      "photo of ukj person",
# #         "class_prompt":         "photo of a person",
# #         "instance_data_dir":    "/content/data/ukj",
# #         "class_data_dir":       "/content/data/person"
# #     }
# ]

# # `class_data_dir` contains regularization images
# import json
# import os
# for c in concepts_list:
#     os.makedirs(c["instance_data_dir"], exist_ok=True)

# with open("concepts_list.json", "w") as f:
#     json.dump(concepts_list, f, indent=4)

In [13]:
# #@markdown Upload your images by running this cell.

# #@markdown OR

# #@markdown You can use the file manager on the left panel to upload (drag and drop) to each `instance_data_dir` (it uploads faster)

# import os
# from google.colab import files
# import shutil

# for c in concepts_list:
#     print(f"Uploading instance images for `{c['instance_prompt']}`")
#     uploaded = files.upload()
#     for filename in uploaded.keys():
#         dst_path = os.path.join(c['instance_data_dir'], filename)
#         shutil.move(filename, dst_path)

In [14]:
# Hugging Face Login
from huggingface_hub import notebook_login

notebook_login()

In [6]:
# ! pip install functorch==0.2.1 ninja bitsandbytes

     |████████████████████████████████| 20.6 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 77.1 MB/s eta 0:00:01
     |████████████████████████████    | 678.7 MB 78.2 MB/s eta 0:00:02    |█                               | 22.7 MB 26.8 MB/s eta 0:00:29     |████                            | 96.6 MB 79.5 MB/s eta 0:00:09     |██████████████▏                 | 344.9 MB 80.0 MB/s eta 0:00:06     |█████████████████████████       | 607.0 MB 73.0 MB/s eta 0:00:03

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 776.3 MB 18 kB/s s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0
    Uninstalling torch-1.12.0:
      Successfully uninstalled torch-1.12.0


# EXP 1: Sunglasses

In [16]:
  #--use_8bit_adam \
    
! accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir="../../outputs/fontresults" \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=3434554 \
  --resolution=512 \
  --train_batch_size=1 \
  --mixed_precision="fp16" \
  --gradient_accumulation_steps=1 \
  --learning_rate=2e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --sample_batch_size=4 \
  --max_train_steps=1500 \
  --save_interval=250 \
  --save_guidance_scale=20 \
  --save_infer_steps=250 \    
  --save_sample_prompt="@sunglasses-jjj beach sunny, bright lighting, sepia tone, california beach" \
  --concepts_list="../../res/concepts_list_sunglasses.json" \
  --not_cache_latents


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--num_cpu_threads_per_process` was set to `30` to improve out-of-box performance
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[!] Not using xformers memory efficient attention.
--------------------------------------------------------------------------
detected:

  Local host:            150-136-46-17
  Device name:           mlx5_0
  Device vendor ID:      0x02c9
  Device vendor part ID: 4126

Default device parameters will be used, which may result in lower
performance.  You can edit any of the files specified by the
btl_openib_device_param_files MCA parameter to set values for your
device.

NOTE: You can turn off this warning by setting the MCA parameter
      btl_openib_warn_no_device_params_found

In [ ]:
  #--use_8bit_adam \
    
! accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir="../../outputs/fontresults-4" \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=3434554 \
  --resolution=512 \
  --train_batch_size=1 \
  --mixed_precision="fp16" \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-7 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=52 \
  --sample_batch_size=4 \
  --max_train_steps=40000 \
  --save_interval=250 \
  --save_sample_prompt="black and white character of the letter b lowercase font icon" \
  --concepts_list="../../res/concepts_list_fonts-4.json" \
  --not_cache_latents


# Convert Weights

In [7]:
# #@markdown Run to generate a grid of preview images from the last saved weights.
# import os
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# weights_folder = OUTPUT_DIR
# folders = sorted([f for f in os.listdir(weights_folder) if f != "0" and '.ipynb' not in f], key=lambda x: int(x))

# row = len(folders)
# col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))
# scale = 4
# fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

# for i, folder in enumerate(folders):
#     folder_path = os.path.join(weights_folder, folder)
#     image_folder = os.path.join(folder_path, "samples")
#     images = [f for f in os.listdir(image_folder) if '.ipynb' not in f]
#     for j, image in enumerate(images):
#         if row == 1:
#             currAxes = axes[j]
#         else:
#             currAxes = axes[i, j]
#         if i == 0:
#             currAxes.set_title(f"Image {j}")
#         if j == 0:
#             currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
#         image_path = os.path.join(image_folder, image)
#         img = mpimg.imread(image_path)
#         currAxes.imshow(img, cmap='gray')
#         currAxes.axis('off')
        
# plt.tight_layout()
# plt.savefig('grid.png', dpi=72)

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [10]:
#@markdown Run conversion.
ckpt_path = WEIGHTS_DIR + "/model.ckpt"

half_arg = ""
#@markdown  Whether to convert to fp16, takes half the space (2GB).
fp16 = True #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
[*] Converted ckpt saved at ../../outputs/fontresults-6/3250/model.ckpt


In [16]:
WEIGHTS_DIR

'../../outputs/fontresults-6/3250'

In [17]:
ckpt_path

'../../outputs/fontresults-6/3250/model.ckpt'

In [19]:
# convert -> .ckpt
for base_dir in np.arange(250, 3500, 250):
    ckpt = '../../outputs/fontresults-6/{}'.format(base_dir)
    
    ckpt_path = ckpt + "/model.ckpt"

    half_arg = ""
    #@markdown  Whether to convert to fp16, takes half the space (2GB).
    fp16 = True #@param {type: "boolean"}
    if fp16:
        half_arg = "--half"
    !python convert_diffusers_to_original_stable_diffusion.py --model_path $ckpt  --checkpoint_path $ckpt_path $half_arg
    print(f"[*] Converted ckpt saved at {ckpt_path}")

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
[*] Converted ckpt saved at ../../outputs/fontresults-6/250/model.ckpt
Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format
[*] Converted ckpt saved at ../../outputs/fontresults-6/500/model.ckp

## Inference

In [11]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")

g_cuda = None

[!] Not using xformers memory efficient attention.


/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [12]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [22]:
#@title Run for generating images.
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

In [20]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

#@title Run for generating images.
def plot_samples_3x3(prompt,
                     model_path,
                     title=None, #@param {title for plot}
                     negative_prompt = "", #@param {type:"string"}
                     num_samples = 9, #@param {type:"number"}
                     guidance_scale = 25, #@param {type:"number"}
                     num_inference_steps = 500, #@param {type:"number"}
                     height = 512, #@param {type:"number"}
                     width = 512, #@param {type:"number"}):
                     seed = 52362,
                     scheduler=None

                        #prompt = "black and white roman soldier icon" #@param {type:"string"}
                    ):  
    
    print('using model_path={}'.format(model_path))
    
    if scheduler is None:
        scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
        
    pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    g_cuda = None
    
    #@markdown Can set random seed here for reproducibility.
    g_cuda = torch.Generator(device='cuda')
    g_cuda.manual_seed(seed)
    

    with autocast("cuda"), torch.inference_mode():
        images = pipe(
            prompt,
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_samples,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=g_cuda
        ).images


    rows=3
    cols = 3
    img_count = 0

    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(15,15))

    for i in range(rows):
        for j in range(cols):        
            if img_count < len(images):
                axes[i, j].imshow(images[img_count])
                img_count+=1
                axes[i, j].axis('off')
                
    if title is not None:
        fig.suptitle(title, fontsize="x-large")
    plt.show()

    # for img in images:
    #     display(img)